# Prerrequisitos 

## Dependencias 



In [ ]:
import jax.numpy as np 
from jax import random 

from inrmri.data_harvard import load_data
from inrmri.data_splitter import SimpleDataLoader
from inrmri.fourier_features import FF_fraction_static_mixed_net
from inrmri.radon_training import spacelim, spoke_loss_fourierspace_phase
from inrmri.loggers import LocalLogger 

from inrmri.basic_nn import simple_train 
import optax 

import matplotlib.pyplot as plt 

## BART Toolbox 

Correr el ejemplo requiere tener instalado el [BART Toolbox](https://mrirecon.github.io/bart/). Ver las instrucciones el el [`README-es`](../README-es.md)

# Obtener datos y generar el conjunto de entrenamiento 

Los datos se obtendrán de [Replication Data for: Multi-Domain Convolutional Neural Network (MD-CNN) For Radial Reconstruction of Dynamic Cardiac MRI](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi%3A10.7910%2FDVN%2FCI3WB6&version=2.0&q=&fileAccess=&fileTag=&fileSortField=&fileSortOrder=). 

## Parámetros para el procesamiento de datos 

- `chosen_patient`:`str` en `"P01"`, `"P02"`, ..., `"P15"`. El código funciona con cualquiera que esté en el diccionario `HARVARD_DB_IDs` de `inrmri.data_harvard`. Es posible añadir pacientes extras añadiendo nuevas keys al diccionario. El _id_ pedido es único a cada paciente y es parte del URL de descarga como un código de 6 valores (letras o números) que está después del _id_ del repositorio (`CI3WB6`). Para [el paciente `P100`](https://dataverse.harvard.edu/file.xhtml?persistentId=doi:10.7910/DVN/CI3WB6/HR66QD&version=2.0) por ejemplo, su _id_ es `HR66QD`.
- `hermitic_fill`:`bool` Este dataset fue adquirido con [_partial echo_](https://www.mr-tip.com/serv1.php?type=db1&dbs=Partial%20Echo) (ver imagen), es decir, los datos radiales no son simétricos en el k-space con respecto al centro. Usar `hermitic_fill = True` completa los datos faltantes.

<p align="center">
  <img src="partial-echo.png" alt="Partial echo" width="300"/>
</p>


- `relaxed_pad_removal`:`bool`. El oversampling usual de un data set radial corresponde a adquirir spokes de tamaño 400. Las de este dataset son de 800; todo los datos adicionales son 0. El preprocesamiento elimina este padding simétricamente (a ambos lados de la _spoke_, de forma que el centro no queda desalineado. Eso es muy importante para que la transformada de Fourier funcione después) de dos formas posibles: el modo _relaxed_ (`relaxed_pad_removal=True`) elimina todos los 0s que puede de forma simétrica sin eliminar ningún dato no nulo. El modo _agresive_ (`relaxed_pad_removal=False`) elimina simétricamente hasta que no quedan 0s en los extremos de la spoke. La diferencia es mínima en el caso en que `hermitic_fill=True`. Cuando `hermitic_fill=False`, el _partial echo_ de los datos hace que la diferencia entre ambos modos sea más significativa. En este caso el k-space generado con _relaxed_ permite tener una imagen de mayor resolución pero que puede ser inconsistente (ciertas spokes dirán que euna zona tiene frecuencias 0, mientras que las spokes con el ángulo contrario dirán que sí hay valores en usa zona). Pero, si recuerdo bien (😂), las reconstrucciones que se obtenían eran razonables igualmente. El modo _agressive_ pierde bastante resolución porque elimina todos los datos "no simétricos". 

> 📌 
> Se recomienda usar `hermitic_fill=True` y `relaxed_pad_removal=False`.

- `sub_spokes_per_frame`:`int`. Relacionado con el factor de aceleración que se desea. Los datos disponible son (casi) totalmente muestreados (usualmente 196 _spokes_ disponibles para reconstruir una imagen de 208x208, lo que da un factor de aceleración muy bajo de $208/196 \approx 1.06$). Pero nos interesa hacer reconstrucciones submuestreadas, es decir, con muchas menos _spokes_ por frame. Usar 16 spokes por frame (`sub_spokes_per_frame=16`) es una cantidad razonable. Usar más debería dar mejores reconstrucciones (pero considerar que el tamaño del _training set_ es directamente proporcional a la cantidad de _spokes_ por frame, por lo que habría que aumentar el número de iteraciones para mantener el mismo número de _epochs_). Yo suelo usar 8, y también es posible usar menos, pero en general deja de tener sentido físico usar factores de aceleración tan grandes (usando menos de 8 spokes por frame, el total de spokes que se adquirirían prospectivamente no alcanza para un ciclo cardíaco completo).
- `tiny_number`:`int`. El dataset fue adquirido con espaciado uniforme (con ángulo de $2\pi/196$ entre una _spoke_ y la siguiente, de forma que al adquirir las 196 se dé una vuelta completa al k-space). Una forma más eficiente es usar submuestreo con [_golden angle_ o _tiny golden angle_](https://onlinelibrary.wiley.com/doi/epdf/10.1002/jmri.28187). Las spokes que se toman del dataset intentan replicar este tipo de submuestreo. El parámetro `tiny_number` permite usar el _tiny golden angle_. 

> 📌 
> Se recomienda usar `tiny_number=1` (correspondiente al _golden angle_) o el mismo valor usado en `sub_spokes_per_frame`.


In [ ]:
random_seed  = 0

key_split, key_B, key_init, key_train = random.split(random.PRNGKey(random_seed), 4)

config_data = {
    'chosen_patient'        : "P02", 
    'hermitic_fill'         : True, 
    'relaxed_pad_removal'   : False,
    'sub_spokes_per_frame'  : 12, 
    'tiny_number'           : 1,
}

## Carga de datos 

A partir del diccionario de configuración, se utiliza la función `load_data` para cargar todo lo necesario. La primera vez que se usa con un paciente descarga el dataset y lo guarda en una carpeta `harvardDB/` que debe haber sido creada previamente. También debe existir `data_bart/`, donde se guardarán las archivos generados por BART: 

- `X_full, Y_full`: conjunto de entrenamiento para la red. Al cargar los datos con `hermitic_fill=True` y `relaxed_pad_removal=False`, la dimensión de _read out_ (el largo de las _spokes_) es de 414. Cada spoke se trata como 1 dato (es decir `batch_size=2` corresponde a 2 spokes). El total de datos entonces es `sub_spokes_per_frame * total_frames`, donde `total_frames` es la cantidad total de frames en que están agrupados los datos. `X_full:array[float]` tiene la información de todas las _spokes_ resultantes luego del submuestreo (la ubicación de las frecuencias adquiridas en el k-space específicamente), además del _frame_ al que están asociadas. `Y_full:array[complex]` tiene las mediciones de cada _spoke_ para cada una de las bobinas.
  - `X_full.shape = (sub_spokes_per_frame * total_frames, 414 + 1, 2)`. El 1 del `414 + 1` es una forma fea de añadir el tiempo de la adquisición a los datos, y el 2 es porque las adquisiciones son 2D en el k-space ($k_x, k_y$).
  - `Y_full.shape = (sub_spokes_per_frame * total_frames, n_coils, 414, 1)`. `n_coils` es el número de bobinas con que se adquirieron los datos. El 1 es porque facilita ajustar los datos con la dimensión de salida de la red.
- `csmap:array[complex]`: mapas de sensibilidad estimados con BART a partir de los datos totalmente muestreados. `csmap.shape = (n_coils, 414, 414)` (el largo de la _spoke_ determina el tamaño de la imagen reconstruida).
- `im:array[complex]`: una reconstrucción de referencia (un vídeo) obtenida a partir de la adquisición totalmente muestreada y GRASP con parámetros $\lambda = 1\times10^{-3}, \mathcal{L} = 5\times 10^{-4}$ y 100 iteraciones. `im.shape = (414,414,total_frames)`. 
- `hollow_mask:array[float]`: usualmente los mapas de sensibilidad tienen zonas donde todas las bobinas tienen sensibilidad 0. Esto impide que red sea capaz de estimar una reconstrucción en esas zonas, porque no tienen ninguna importancia en la función de pérdida. Esto en general no importa, ya que esas zonas suelen estar fuera del cuerpo, así que es razonable que la estimación allí sea 0. En la práctica, se permite que la red prediga cualquier cosa en esas zonas y luego se usa la máscara para cubrirlas con 0s. `hollow_mask.shape = (414,414)` y *marca las zonas en las que no se pueden reconstruir*, es decir, vale 1 donde no se puede optimizar la red (la zona que se va a enmascarar posteriormente) y 0 donde sí se puede optimizar.


La función `get_splitted_dataset` me permite dejar algunas _spokes_ como conjunto de validación.

In [ ]:
X_full, Y_full, csmap, im, hollow_mask = load_data(config_data)

loader = SimpleDataLoader(X_full, Y_full)

training_fraction = 0.7 

X_train, Y_train, X_val, Y_val = loader.get_splitted_dataset(key_split, training_fraction)

trainset = (X_train, Y_train)
valset = (X_val, Y_val)

In [ ]:
# Visualizar la reconstrucción de referencia, los mapas de sensibilidad y la máscara

chosen_csmap = 12 
chosen_frame = 15 
fig = plt.figure(figsize=(15,5))

plt.subplot(131)
plt.imshow(np.abs(im[...,chosen_frame]), cmap='bone')
plt.title(f"Reconstrucción completamente, \nmuestreada en el {chosen_frame}-ésimo frame")

plt.subplot(132)
plt.imshow(np.abs(csmap[chosen_csmap]))
plt.title(f"{chosen_csmap}-ésimo mapa de sensibilidad")

plt.subplot(133)
plt.imshow(hollow_mask, cmap='Grays')
plt.title(f"Zonas donde la reconstrucción \n de la red está indeterminada")


## Cargar la red 

Por simplicidad voy a elegir la red tipo `FF_fraction_static_mixed_net`, que es la que ha dado mejores resultados. Sin embargo, la función `add_FFgroup_to_parser` permite hacer _parsing_ automático de la red a usar con un `ArgumentParser`, lo que es especialmente útil para elegirla fácilmente en scripts.

- `sigma:float`: parámetro de Fourier Features, relacionado con las frecuencias que la red aprende más rápidamente. Para este dataset, usualmente valores de $\sigma \in [3, 12]$ dan buenos resultados.
- `radon_seed:int`: semilla aleatoria (_random seed_) para tener aleatoriedad reproducible
- `frac_static_mixed:(float,int)`: El primer valor $p_s \in [0,1]$ está asociado a la regularización temporal: mientras más cercano a 1, mayor regularización. El segundo valor $L$ es el largo del vector de Fourier Features. En general, mientras más grande, más robusto se hacen los resultados al valor de `sigma`, pero también requiere más memoria.

In [ ]:
sigma  =  7.5

frac_static_mixed_params = (0.8, 1000) 

FFnet = FF_fraction_static_mixed_net(*frac_static_mixed_params, sigma, im, key_B)

## Definir la función de pérdida 

- `freq_filter`:`str` en `'ramp'`, `'cosine'`, `'shepp-logan'`. Se encontró que ponderar con un mayor peso las altas frecuencias en la función de pérdida aceleraba la convergencia de la red. Sin embargo, [es usual que la transformada de Radon aplique una _filtered backprojection_](https://scikit-image.org/docs/stable/auto_examples/transform/plot_radon_transform.html#reconstruction-with-the-filtered-back-projection-fbp), para eliminar el ruido producido por las altas frecuencias (ver imagen de varios posibles filtros). En nuestro trabajo consideramos el peso $(1 + |k|)$ en la función de pérdida, que llamamos modo `'ramp'` (notar que no es igual al de la figura). También se han probado `'cosine'` y `'shepp-logan'` sin diferencias significativas (hasta unas 10k), por lo que se recomienda el uso de `'ramp'`.

<p align="center">
  <img src="high-freq-filters.png" alt="High frequency filters" width="500"/>
</p>


In [ ]:
spclim = spacelim(X_train[:,1:,:]) # esto era para definir el dominio (en el kspace o en imagen, ya no me acuerdo)

# nframes = im.shape[2]

from jax import jit, vmap 

freq_filter = 'ramp'

short_spoke_loss = lambda params, train_X_sample, train_Y_sample: spoke_loss_fourierspace_phase(params, train_X_sample, train_Y_sample, spclim, 1., csmap, FFnet, freq_filter)
radon_fourier_loss_fourierspace_phase = jit(lambda params, train_X, train_Y: np.mean(vmap(short_spoke_loss, in_axes = (None, 0, 0))(params, train_X, train_Y)))
loss = lambda params, train_X, train_Y: radon_fourier_loss_fourierspace_phase(params, train_X, train_Y) 


## Entrenar la red 

In [ ]:
folder = './results'

inner_layers = [512, 512, 512]

nIter = 1000 # 1k para probar, pero usualmente se necesita 10k, y sigue mejorando un poco hasta 20k
batch_size = 1 # lo maximo para una gpu de 10gb con esta configuracion de inner_layer y frac_static_mixed_params[1]
learning_rate = 1e-3 # TODO: ver cual usé 

params = FFnet.init_params(inner_layers, key = key_init)

# args_dic['nframes'] = nframes
# args_dic['layers'] = inner_layers
# args_dic['ff-type'] = FFnet.ff_type()

# logger = LocalLogger(folder)

# config = {}

# logger_init_params = {
#     "project" : "invivo-jupyter-nb",
#     "group"   : "",
#     "config"  : 
# }

optimizer = optax.adam(learning_rate)

results = simple_train(loss, *trainset, params,  optimizer, key_train, batch_size = batch_size, nIter = nIter)

In [ ]:
results.keys()

## Revisar resultados 

In [ ]:
plt.plot(results['train_loss'])
plt.yscale('log')
plt.title('Evolución de la _loss_')

In [ ]:
frame = 8 

# @jit
# def image_pred(params, frame):
    # return np.abs(FFnet.image_prediction_at_timeframe(params, FFnet._gridt[frame]) * (1 - hollow_mask) * is_inside) * is_inside

reco = FFnet.image_prediction_at_timeframe(results['last_param'], FFnet._gridt[frame]) # shape (414,414)

In [ ]:
fig = plt.figure(figsize=(15,5))

plt.subplot(121)
plt.imshow(np.abs(im[...,frame]), cmap='bone')
plt.title(f"Reconstrucción completamente, \nmuestreada en el {frame}-ésimo frame")

plt.subplot(122)
plt.imshow(np.abs(reco), cmap='bone')
plt.title(f"Reconstrucción Neural Fields")

## Calcular métricas 

## Métricas disponibles 

El diccionario `METRIC_FUNCTIONS` de `inrmri.metrics` tiene varias métricas implementadas. 
metrics_values 
- `'ssim_3D'`: a pesar del nombre, esta es la SSIM estándar, calculada sobre la imagen completa (como suelo trabajar con vídeos, mis imágenes son 3D y la llamada así para distinguirla de la siguiente)
- `'mean_ssim_2D'`: calcula la SSIM frame por frame y la promedia. Asume que las imagenes tienen shape `(px, py, total_frames)`.
- `'psnr'`
- `'ism_fsim'`, `'ism_issm'`, `'ism_psnr'`, `'ism_rmse'`: están directamente extraídas del paquete [https://pypi.org/project/image-similarity-measures/](https://pypi.org/project/image-similarity-measures/). No sé por qué `'ism_fsim'` no está funcionando con imágenes 2D.

## Preprocesamiento 

Suelo calcular las métricas en torno al corazón. La variable `PATIENTS_CROPS` de `inrmri.data_harvard` tiene calculadas las zonas adecuadas para varios pacientes. El preprocesamiento, que suele incluir corte, normalización y cálculo del valor absoluto, se realiza con `BeforeNormalizer` o `BeforeLinRegNormalizer`. El primero normaliza por la máxima intensidad del píxel en la zona cortada, el segundo escala la imagen por un factor adecuado de forma que maximize el PSNR, por lo que requiere la imagen de referencia.

In [ ]:
from inrmri.metrics import METRIC_FUNCTIONS 

print("The available metrics are: " + str(list(METRIC_FUNCTIONS.keys())))

from inrmri.image_processor import BeforeLinRegNormalizer, BeforeNormalizer 
from inrmri.data_harvard import PATIENTS_CROPS 

crop_ns = PATIENTS_CROPS[config_data['chosen_patient']]
improc = BeforeLinRegNormalizer(im[...,frame], crop_ns)
# improc = BeforeNormalizer(crop_ns)

chosen_metrics = ['ssim_3D', 'psnr'] # no todas funcionan en 2D 

processed_reco = improc.process(reco)
processed_ref = improc.process(im[...,frame])

metrics_values = {
    metric_name: METRIC_FUNCTIONS[metric_name](processed_ref, processed_reco) for metric_name in chosen_metrics
}

metrics_values 

In [ ]:

fig = plt.figure(figsize=(10,5))

plt.subplot(121)
plt.imshow(processed_ref, cmap='bone')
plt.title(f"Reconstrucción completamente, \nmuestreada en el {frame}-ésimo frame \n en la zona del corazón")

plt.subplot(122)
plt.imshow(processed_reco, cmap='bone')
metrics_str = " ".join([f"\n{metric_name} {metric_val:.2f}" for metric_name, metric_val in metrics_values.items()])
plt.title(f"Reconstrucción Neural Fields" + metrics_str)